In [7]:
# Useful starting lines
%matplotlib inline
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# import scikit-surprise stuff
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [3]:
# import costum stuff
from utility import *

In [4]:
# prepare file in correct format for scikit-surprise
filepathcsv = "../data/data_train.csv"
filepath = "../data/data_train.dat"
convertCSV2Surprise(filepathcsv,filepath)

In [5]:
# import data
csvreader = Reader(line_format=u"user item rating",sep=";",rating_scale=(1,5),skip_lines=1)
data = Dataset.load_from_file(filepath,csvreader)

# chose algorithm
algo = SVD()

In [8]:
# test sequential execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=5, verbose=True)
t = time.clock()-t
print("Sequential execution time: ",t)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0317  1.0265  1.0251  1.0245  1.0273  1.0270  0.0025  
MAE (testset)     0.8229  0.8192  0.8172  0.8177  0.8200  0.8194  0.0020  
Fit time          65.34   62.75   63.43   63.59   64.10   63.84   0.87    
Test time         3.02    2.99    3.06    3.00    3.32    3.08    0.12    
Sequential execution time:  347.610254


In [9]:
# testing parallel execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=5, verbose=True,n_jobs=-1)
t = time.clock()-t
print("Parallel execution time: ",t)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0252  1.0296  1.0275  1.0282  1.0286  1.0278  0.0015  
MAE (testset)     0.8187  0.8223  0.8209  0.8208  0.8212  0.8208  0.0012  
Fit time          75.37   91.16   104.30  90.57   63.60   85.00   14.09   
Test time         4.14    4.75    4.47    2.63    2.31    3.66    1.00    
Parallel execution time:  264.694266


In [29]:
GridSearchCV?

In [27]:
print([i for i in range(75,101,25)])

[75, 100]


In [32]:
grid = \
{\
"n_factors": [i for i in range(75,101,25)],\
#"lr_all": [0.005,0.01],\
#"reg_all": [0.02,0.04],\
"n_epochs": [10,20]
}

gridsearch = GridSearchCV(algo_class=SVD,param_grid=grid,measures=['RMSE','MAE'], cv=2, n_jobs=-1)

In [34]:
gridsearch.fit(data)
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])

1.0102086653143898
{'n_factors': 75, 'n_epochs': 10}


TODO:

* GridSearchCV

* build_full_trainset + algo.predict() -> kaggle

* test other algorithms

* code export to .csv

* manual cross-validation with iterators

* manual split

* maybe build costum prediction algorithm?

* write intro (Netflix prize, Simon Fuchs, other applications,scikit-surpise (why reinvent the wheel?), short summary of content)

* important: in report show that we did understand the different methods!